In [4]:
#import
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import pandas as pd
import numpy as np
import pymysql
import operator
import os
import math
import scipy
from scipy import stats
import statsmodels
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from datetime import *
import scipy.io as sio
import h5py
import copy
import shutil
import rarfile
import random
#from general_function import *
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
sns.set(style="whitegrid")
#myfont = FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf', size=10)
#sns.set(font=myfont.get_name(),style="whitegrid",palette="muted",color_codes=True)

In [2]:
#data
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
dateparse = lambda dates: pd.datetime.strptime(str(date.fromordinal(int(dates)-366)).replace('-',''), '%Y%m%d')
zzIcData = pd.read_csv('zz500_from_db.csv',index_col = 0, date_parser=dateparse)
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y%m%d')
open_p = pd.read_csv('open.csv',index_col = 0, date_parser=dateparse)
close_p = pd.read_csv('close.csv',index_col = 0, date_parser=dateparse)
amt = pd.read_csv('amt.csv',index_col = 0, date_parser=dateparse)
adjfactor = pd.read_csv('adjfactor.csv',index_col = 0, date_parser=dateparse)
open_p.columns = [int(i) for i in open_p.columns]
close_p.columns = [int(i) for i in close_p.columns]
amt.columns = [int(i) for i in amt.columns]
adjfactor.columns = [int(i) for i in adjfactor.columns]
adjopen = adjfactor * open_p
adjclose = adjfactor * close_p
adjpreclose = adjclose.shift(1)
close_to_close = np.log(adjclose/adjpreclose)
open_to_close = np.log(adjclose/adjopen)
preclose_to_open = np.log(adjopen/adjpreclose)
# close_to_close = adjclose/adjpreclose - 1
# open_to_close = adjclose/adjopen - 1
# preclose_to_open = adjopen/adjpreclose - 1
amt = amt.replace(0,np.nan)
amt = amt.T.dropna(how='all').T
def get_rid_of_newly_ipos(df):
    return df.iloc[df.index.tolist().index(df[pd.notnull(df)].index[0])+42:]
amt = amt.apply(get_rid_of_newly_ipos)
close_to_close = close_to_close[pd.notnull(amt)].dropna(how='all')
open_to_close = open_to_close[pd.notnull(amt)].dropna(how='all')
preclose_to_open = preclose_to_open[pd.notnull(amt)].dropna(how='all')
zz500 = (zzIcData['5']/(zzIcData['5'].shift(1).fillna(1000)) - 1)
zz500.name = 'zz500'

In [3]:
def future_value_df_generator(var_list,ret_list,tw = 100):
    var_list.name = 'VaR'
    var_to_test_data = pd.DataFrame(var_list.dropna())
    for i in range(tw):
        var_to_test_data[i] = np.nan
    for i in var_to_test_data.index:
        test_data = ret_list.iloc[ret_list.index.tolist().index(i)+1:].iloc[:tw].reset_index(drop=True)
        if len(test_data) != tw:
            break
        for j in range(tw):
            var_to_test_data.loc[i,j] = test_data.loc[j]
    var_to_test_data.dropna(inplace=True)
    return var_to_test_data
def count_exception(df, tw):
    VaR = df.loc['VaR']
    test_data = df.loc[range(tw)]
    return len(test_data[test_data<VaR])
def kupiec_backtest(df, cl, tw):
    cl_a = 1 - cl
    LRuc_left = -2*np.log((cl**(tw - df.loc['n_exception'])) * (cl_a**df.loc['n_exception']))
    LRuc_right = 2*np.log(((1 - df.loc['n_exception']/tw)**(tw - df.loc['n_exception']) *\
                           ((df.loc['n_exception']/tw)**df.loc['n_exception'])))
    return LRuc_left + LRuc_right
def christoffersen_test(df, tw):
    VaR = df.loc['VaR']
    n_exception = df.loc['n_exception']
    if n_exception == 0 or (n_exception == 1 and df.loc[tw-1] < VaR):
        return 0
    test_data = df.loc[range(tw)].tolist()
    n_00 = len([value for index,value in enumerate(test_data[:-1]) if value > VaR and test_data[index+1] > VaR])
    n_10 = len([value for index,value in enumerate(test_data[:-1]) if value < VaR and test_data[index+1] > VaR])
    n_01 = len([value for index,value in enumerate(test_data[:-1]) if value > VaR and test_data[index+1] < VaR])
    n_11 = len([value for index,value in enumerate(test_data[:-1]) if value < VaR and test_data[index+1] < VaR])
    pi_0 = n_01 / (n_00 + n_01) if n_00 + n_01 != 0 else 0
    pi_1 = n_11 / (n_10 + n_11) if n_10 + n_11 != 0 else 0
    pi = (n_01 + n_11) / (n_00 + n_01 + n_10 + n_11)
    LRcc_up = ((1 - pi)**(n_00 + n_10)) * (pi**(n_01+n_11))
    LRcc_down = ((1 - pi_0)**n_00) * (pi_0**n_01) * ((1 - pi_1)**n_10) * (pi_1**n_11)
    LRcc = -2*np.log(LRcc_up / LRcc_down)
    return LRcc
def exception_test(df, cl):
    LRuc = df.loc['LRuc']
    LRcc = df.loc['LRcc']
    #uc - chi-sqaure 1 df
    #cc - chi-square 2 df
    LRuc_limit = stats.chi2.ppf(cl, df = 1)
    LRcc_limit = stats.chi2.ppf(cl, df = 2)
    if LRcc > LRcc_limit:
        if LRuc + LRcc > LRuc_limit + LRcc_limit:
            return 'reject'
        else:
            return 'accept'
    else:
        if LRuc > LRuc_limit:
            return 'reject'
        else:
            return 'accept'
def backtest_VaR(ret_data, var_data, window = 100, var_cl = 0.95, test_cl = 0.95):
    var_data.name = 'VaR'
    var_confidence_level = var_cl
    test_confidence_level = test_cl
    data =  future_value_df_generator(var_data,ret_data,tw = window)
    data['n_exception'] = data.apply(count_exception,args=(window,),axis=1)
    data['LRuc'] = data.apply(kupiec_backtest,args=(var_confidence_level,window,),axis=1)
    data['LRcc'] = data.apply(christoffersen_test,args=(window,),axis=1)
    data['result'] = data.apply(exception_test,args=(test_confidence_level,),axis=1)
    return data

In [9]:
def original_VaR(df, cl):
    cl_a = 1 - cl
    return sorted(df)[int(len(df)*cl_a)]

In [10]:
backtest_VaR(close_to_close[2072].dropna(), \
             close_to_close[2072].dropna().rolling(window = 100).apply(original_VaR,args=(0.99,)), \
             window = 6, var_cl = 0.99, test_cl = 0.95)['result'].value_counts(normalize=True)

accept    0.894098
reject    0.105902
Name: result, dtype: float64

In [ ]:
2148/(2148+131)

In [ ]:
from scipy import stats

In [ ]:
stats.probplot((full_oc.xs(1,level=0)['open_to_close'].dropna().sort_values().tolist()),dist="norm", plot=plt)
plt.show()

In [ ]:
stats.probplot(pd.Series(np.log((full_oc.xs(1,level=0)['open_to_close']/100+1).dropna().sort_values().tolist())),\
    dist="lognorm", sparams = (0.02,), plot=plt)
plt.show()

In [ ]:
stats.probplot(full_oc.xs(1,level=0)['open_to_close'].dropna().sort_values().tolist(), dist="t", sparams = (5,), plot=plt)
plt.show()

In [ ]:
#Age weighted VaR

In [ ]:
test_window = 100
var_confidence_level = 0.95
test_confidence_level = 0.95

In [ ]:
exp = full_cc.xs(1,level=0)['close_to_close'].dropna().iloc[-700:]

In [ ]:
def original_VaR(df, cl):
    cl_a = 1 - cl
    return sorted(df)[int(len(df)*cl_a)]
exp_var = exp.rolling(window = test_window).apply(original_VaR,args=(var_confidence_level,))

In [ ]:
backtest = backtest_VaR(exp,exp_var,window = 50)

In [ ]:
backtest['result'].value_counts()/len(backtest)

In [ ]:
param_lambda = 0.9
param_max_length = 100

def age_weighted_VaR(df, cl, ld):
    weights = [(ld**(i-1))*(1-ld)/(1-ld**len(df)) for i in range(len(df))][::-1]
    mid = pd.DataFrame([df,weights]).T.sort_values(0)
    mid[2] = mid[1].cumsum()
    cl_a = 1 - cl
    return mid[0].iloc[next(index for index, value in enumerate(mid[2]) if value > cl_a)]

exp_var = exp.rolling(window = param_max_length, min_periods = test_window).apply(age_weighted_VaR,args=(var_confidence_level,param_lambda,))

In [ ]:
backtest = backtest_VaR(exp,exp_var,window = 50)

In [ ]:
backtest['result'].value_counts()/len(backtest)

In [ ]:
#GARCH model to predict volatility

In [ ]:
from scipy import  stats
import statsmodels.api as sm  # 统计相关的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arch  # 条件异方差模型相关的库

In [ ]:
data = full_cc.xs(1,level=0)['close_to_close'].dropna().iloc[-700:]

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(data.tolist())
plt.show()

In [ ]:
t = sm.tsa.stattools.adfuller(data)  # ADF检验
print("p-value:   ",t[1])

In [ ]:
fig = plt.figure(figsize=(20,5))
ax1=fig.add_subplot(111)
fig = sm.graphics.tsa.plot_pacf(data,lags = 20,ax=ax1)

In [ ]:
n_lags = 11
n_test = 100

train = data[:-n_test]
test = data[-n_test:]
am = arch.arch_model(train,mean='AR',lags=n_lags,vol='GARCH') 
res = am.fit()

In [ ]:
res.summary()

In [ ]:
ini = res.resid[-n_lags:]
a = np.array(res.params[1:-3])
w = a[::-1] # 系数
for i in range(n_test):
    new = test.iloc[i] - (res.params.iloc[0] + w.dot(ini[-n_lags:]))
    ini = np.append(ini,new)
at_pre = ini[-n_test:]
at_pre2 = at_pre**2

In [ ]:
ini2 = np.array(res.conditional_volatility[-2:])

for i in range(n_test):
    new = res.params.loc['omega'] + res.params.loc['alpha[1]']*at_pre2[i] + res.params.loc['beta[1]']*ini2[-1]
    ini2 = np.append(ini2,new)
vol_pre = ini2[-n_test:]

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(data.tolist(),label='origin_data')
plt.plot(res.conditional_volatility.tolist(),label='conditional_volatility')
x=range(len(train),len(train)+n_test)
plt.plot(x,vol_pre,'r-',label='predict_volatility')
plt.legend(loc=0)

In [ ]:
reg = OLS(data.diff(1).dropna().tolist(),[[1,i] for i in data.shift(1).loc[data.diff(1).dropna().index].tolist()])

In [ ]:
result = reg.fit()

In [ ]:
result.summary()

In [ ]:
plt.plot(data.shift(1).loc[data.diff(1).dropna().index],data.diff(1).dropna().tolist(),'r.')
plt.plot(np.arange(-10,10,0.01)*result.params[1],np.arange(-10,10,0.01),'b-')
plt.show()

In [ ]:
am = arch.arch_model(data,mean='AR',lags=5,vol='GARCH',p = 1, q = 1)
index = data.index
start_loc = 0
end_loc = 200
forecasts = {}
for i in range(len(data)-end_loc):
    sub_res = am.fit(first_obs=i, last_obs=i+end_loc, disp='off')
    temp = sub_res.forecast(horizon=1,method = 'bootstrap',simulations = 10).variance
    fcast = temp.iloc[i+end_loc-1]
    forecasts[fcast.name] = fcast
forecasted_vol = pd.DataFrame(forecasts).T**(1/2)
conditional_vol = res.conditional_volatility.loc[pd.DataFrame(forecasts).T.index]
rolling_std = data.rolling(window=100).std().loc[conditional_vol.index]
plt.plot(pd.ewma(forecasted_vol['h.1'],span = 1).tolist(),'r-')
plt.plot(rolling_std.tolist(),'g-')
plt.show()

In [ ]:
#Volatility-adjusted VaR

In [ ]:
vaV = pd.concat([data,rolling_std,pd.ewma(forecasted_vol['h.1'],span = 1)],axis=1,join='inner')
vaV.columns = ['ret','std','forecasted_vol']

In [ ]:
pd.set_option('display.max_rows',20)
pd.set_option('display.max_columns',20)

In [ ]:
vaV

In [ ]:
index_series = []
VaR_series = []
param_lambda = 0.94
param_max_length = 100
VaR = None
old_VaR = None
for i in range(len(vaV)):
    sub_vaV = vaV.iloc[:i+1].copy()
    if len(sub_vaV) < test_window:
        continue
    if len(sub_vaV) <= param_max_length:
        pass
    else:
        sub_vaV = sub_vaV.iloc[-param_max_length:]
        
    #newest_forecasted_vol = sub_vaV.iloc[-1,2]
    newest_forecasted_vol = sub_vaV.iloc[-1,1]
    
    sub_vaV['adjusted_ret'] = sub_vaV['ret']*((newest_forecasted_vol/sub_vaV['std']))
    index_series.append(sub_vaV.index.tolist()[-1])
    
    bootstrap_factor = 0.6
    VaR_sub_list = []
    for k in range(10):
        selecting = random.sample([j for j in range(len(sub_vaV))],int(len(sub_vaV)*bootstrap_factor))
        sub_sub_vaV = sub_vaV.iloc[selecting]
        sub_sub_vaV['weights'] = [(param_lambda**(j-1))*(1-param_lambda)/(1-param_lambda**len(sub_sub_vaV)) \
            for j in range(len(sub_sub_vaV))][::-1]
        sub_sub_vaV = sub_sub_vaV.sort_values('adjusted_ret')
        sub_sub_vaV['weights_cumsum'] = sub_sub_vaV['weights'].cumsum()
    
        VaR = sub_sub_vaV['adjusted_ret'].iloc[next(index for index, value in enumerate(sub_sub_vaV['weights_cumsum']) \
                                                        if value > (1-var_confidence_level))]
        VaR_sub_list.append(VaR)
    VaR = np.mean(VaR_sub_list)

    #sub_vaV['weights'] = [(param_lambda**(j-1))*(1-param_lambda)/(1-param_lambda**len(sub_vaV)) \
    #    for j in range(len(sub_vaV))][::-1]
    #sub_vaV = sub_vaV.sort_values('adjusted_ret')
    #sub_vaV['weights_cumsum'] = sub_vaV['weights'].cumsum()
    # 
    #VaR = sub_vaV['adjusted_ret'].iloc[next(index for index, value in enumerate(sub_vaV['weights_cumsum']) \
    #                                                    if value > (1-var_confidence_level))]
    
    if old_VaR == None:
        old_VaR = VaR
        VaR_series.append(VaR)
        continue
    else:
        #if VaR < 1.1 * old_VaR:
        #    VaR = 1.1 * old_VaR
        #elif VaR > 0.99 * old_VaR:
        if VaR > 0.99 * old_VaR:
            VaR = 0.99 * old_VaR
    old_VaR = VaR
    VaR_series.append(VaR)
exp_var = pd.Series(VaR_series,index=index_series)

exp_var = pd.ewma(exp_var,span = 20)

In [ ]:
backtest = backtest_VaR(data, exp_var, window = 50, var_cl = 0.95, test_cl = 0.95)

In [ ]:
backtest['result'].value_counts()/len(backtest)

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(111)
ax1.plot(backtest['VaR'].tolist())
ax1.set_ylabel('VaR')

ax2 = ax1.twinx()  # this is the important function
ax2.plot(backtest['n_exception'].tolist(),'r.')
ax2.hlines(0.5,0,len(backtest))
ax2.hlines(6.5,0,len(backtest))
ax2.set_ylabel('n_exception')
plt.show()
plt.plot((data.loc[backtest.index]/100+1).cumprod().tolist())
plt.ylabel('stock')
plt.show()

Quasi-CIR (Cox Ingersoll Ross) model for a volatility process

In [ ]:
while(True):
    stock = random.choice(full_cc.index.levels[0].tolist())
    data = full_cc.xs(stock,level=0)['close_to_close'].dropna()
    data = data[(data<10.1) & (data>-10.1)]
    if len(data) <= 601:
        print('too short')
        continue
    else:
        print(stock,'with length:',len(data))
        amt_data = full_amt.xs(stock,level=0)['amt'].loc[data.index]
        std_data = data.rolling(window=100).std().dropna()
        #std_data = pd.Series([(i - std_data.min())/(std_data.max() - std_data.min())*10 for i in std_data],index=std_data.index)

        fig = plt.figure(figsize=(16,6))

        ax1 = fig.add_subplot(211)
        ax1.plot(std_data.dropna().tolist())

        ax2 = ax1.twinx()  # this is the important function
        ax2.bar(range(len(amt_data.loc[std_data.dropna().index])),amt_data.loc[std_data.dropna().index].tolist(),fc='r',alpha = 0.5)

        ax3 = fig.add_subplot(212)
        ax3.plot(data.loc[std_data.dropna().index].tolist())

        plt.show()
        
        corr_calc_data_std = (std_data.dropna()/std_data.dropna().shift(50) - 1).dropna()
        corr_calc_data_amt = (amt_data.loc[std_data.dropna().index]/amt_data.loc[std_data.dropna().index].shift(1) - 1).dropna()
        corr_calc_data_amt = corr_calc_data_amt.loc[corr_calc_data_std.index].dropna()
        corr_calc_data_std = corr_calc_data_std.loc[corr_calc_data_amt.index]
        
        print(np.corrcoef(corr_calc_data_std,corr_calc_data_amt))
        print(stats.kendalltau(std_data.dropna().tolist()[50:],amt_data.loc[std_data.dropna().index].tolist()[:-50]).correlation)
        
        plt.figure(figsize=(10,3))
        plt.plot(data.shift(1).loc[data.diff(1).dropna().index],data.diff(1).dropna().tolist(),'r.')
        plt.title('return')
        plt.show()
        
        plt.figure(figsize=(10,3))
        plt.plot(std_data.shift(1).loc[std_data.diff(1).dropna().index],std_data.diff(1).dropna().tolist(),'r.')
        plt.title('volatility')
        plt.show()
        
        plt.figure(figsize=(10,3))
        plt.plot((std_data/std_data.shift(1) - 1).shift(1).loc[(std_data/std_data.shift(1) - 1).diff(1).dropna().index],\
                 (std_data/std_data.shift(1) - 1).diff(1).dropna().tolist(),'r.')
        plt.title('delta volatility')
        plt.show()
        break

In [ ]:
all_std_data = []
all_std_diff_data = []
for stock in full_cc.index.levels[0].tolist():
    data = full_cc.xs(stock,level=0)['close_to_close'].dropna()
    data = data[(data<10.1) & (data>-10.1)]
    amt_data = full_amt.xs(stock,level=0)['amt'].loc[data.index]
    amt_data = amt_data.replace(0,np.nan).dropna()
    data = data.loc[amt_data.index]
    if len(data) <= 100:
        continue
    else:
        std_data = data.rolling(window=100).std()
        x = std_data.shift(1).loc[std_data.diff(1).dropna().index].tolist()
        y = std_data.diff(1).dropna().tolist()
        all_std_data += x
        all_std_diff_data += y

In [ ]:
plt.plot(all_std_data,all_std_diff_data,'r.',alpha=0.02)
plt.show()

In [ ]:
volatility_change_data = pd.concat([pd.Series(all_std_data),pd.Series(all_std_diff_data)],axis=1).sort_values(0,ascending=False)\
    .reset_index(drop=True)
volatility_change_data.columns = ['volatility','delta']

In [ ]:
volatility_change_data

In [ ]:
interval_to_data = {}
minimum_number_in_each_interval = 1000
volatility_data_max = volatility_change_data['volatility'].max()
volatility_data_min = volatility_change_data['volatility'].min()
current_upper = volatility_data_max
while(True):
    if current_upper <= volatility_data_min:
        break
    for interval in [0.001,0.005,0.01,0.05,0.1,0.5,1,1.5,2]:
        sub_volatility_change_data = volatility_change_data[(volatility_change_data['volatility'] > current_upper-interval) & \
                                                            (volatility_change_data['volatility'] <= current_upper)]
        if len(sub_volatility_change_data) >= minimum_number_in_each_interval:
            break
    interval_to_data[(current_upper,max(current_upper-interval,volatility_data_min-1e-7))] = sub_volatility_change_data
    current_upper -= interval

In [ ]:
n_interval = 100
interval_to_percentage_distribution = {}
for i in interval_to_data.keys():
    sub_data_series = interval_to_data[i]['delta'].sort_values()
    percentage_distribution = []
    for j in range(n_interval):
        if j == n_interval - 1:
            percentage_distribution.append(sub_data_series.iloc[-(len(sub_data_series)//n_interval):].mean())
        else:
            percentage_distribution.append(sub_data_series.iloc[j*(len(sub_data_series)//n_interval):\
                                                               (j+1)*(len(sub_data_series)//n_interval)].mean())
    interval_to_percentage_distribution[i] = percentage_distribution
interval_to_percentage_distribution_df = pd.DataFrame(interval_to_percentage_distribution)

In [ ]:
interval_to_percentage_distribution_df

In [ ]:
while(True):
    stock = random.choice(full_cc.index.levels[0].tolist())
    stock = 603389
    data = full_cc.xs(stock,level=0)['close_to_close'].dropna()
    if len(data) <= 101:
        
        print('too short')
        continue
    else:
        cut = random.randint(100,len(data)-200)
        print(stock)
        print(cut)
        std_data = data.rolling(window=100).std().dropna()
        
        new_std_data = std_data.iloc[-cut:]
        old_std_data = std_data.iloc[:-cut]
        all_std_data = old_std_data.dropna().copy()
        data = data.loc[std_data.index]
        all_std_data.index = range(-len(all_std_data),0)
        all_std_data = all_std_data.append(new_std_data.reset_index(drop=True))
        old_std_data = old_std_data.dropna().tolist()
        sigma = np.std(old_std_data)*np.sqrt(252)
        m = 0.2
        theta_mean = np.mean(old_std_data)
        theta_min = max(np.min(old_std_data) - m * (np.max(old_std_data) - np.min(old_std_data)), volatility_data_min)
        theta_max = min(np.max(old_std_data) + m * (np.max(old_std_data) - np.min(old_std_data)), volatility_data_max)
        k = 2.5
        fig = plt.figure(figsize=(18,10))
        ax1 = fig.add_subplot(211)
        ax1.plot(all_std_data)
        ending_vol = []
        forecast_days = 50
        ax1.hlines(theta_mean,all_std_data.index.min(),forecast_days,'g')
        ax1.hlines(theta_max,all_std_data.index.min(),forecast_days,'r')
        ax1.hlines(theta_min,all_std_data.index.min(),forecast_days,'r')
        for j in range(100):
            V_0 = new_std_data.iloc[0]
            V_n1 = old_std_data[-1]
            V_list = [V_n1,V_0]
            dt = 1/252

            for i in range(forecast_days):
                V_old_old = V_list[-2]
                V_old = V_list[-1]

                choice = random.randint(0,1)
                jump = random.randint(0,99)

                m = 0.2
                theta_min = max(np.min(old_std_data + V_list) - m * (np.max(old_std_data + V_list)\
                                                                     - np.min(old_std_data + V_list)), volatility_data_min)
                theta_max = min(np.max(old_std_data + V_list) + m * (np.max(old_std_data + V_list)\
                                                                     - np.min(old_std_data + V_list)), volatility_data_max)

                dist_to_max = (theta_max - max(V_old,theta_min))/(theta_max - theta_min)
                dist_to_min = (min(V_old,theta_max) - theta_min)/(theta_max - theta_min)
                dist_max_mid = (theta_max - theta_mean)/(theta_max - theta_min)
                dist_min_mid = (theta_mean - theta_min)/(theta_max - theta_min)

                if choice == 0:
                    V_new = V_old +\
                        np.abs(dist_to_max - dist_max_mid) * np.abs(dist_to_min - dist_min_mid) * (theta_mean - V_old) * dt * 4 * k+\
                        (dist_to_max * dist_to_min) * (V_old - V_old_old) * dt * 4 * k +\
                        sigma * (dist_to_max * dist_to_min * 100) * V_old * dt * k * k * dt
                else:
                    V_new = V_old +\
                        np.abs(dist_to_max - dist_max_mid) * np.abs(dist_to_min - dist_min_mid) * (theta_mean - V_old) * dt * 4 * k +\
                        (dist_to_max * dist_to_min) * (V_old - V_old_old) * dt * 4 * k -\
                        sigma * (dist_to_max * dist_to_min * 100) * V_old * dt * k * k * dt
                
                scaled_V_old = volatility_data_min + dist_to_min * (volatility_data_max - volatility_data_min)
                
                if jump == 0:
                    jump_value = interval_to_percentage_distribution_df[next(col for col in interval_to_percentage_distribution_df\
                        .columns if col[0]>scaled_V_old)].loc[0]
                elif jump == 1:
                    jump_value = interval_to_percentage_distribution_df[next(col for col in interval_to_percentage_distribution_df\
                        .columns if col[0]>scaled_V_old)].loc[n_interval-1]
                else:
                    jump_value = 0

                V_new += jump_value

                V_list.append(V_new)
            ending_vol.append(V_new)
            ax1.plot(V_list,alpha=0.1)
        ax1.plot(0,V_0,'b*')
        ax1.plot(forecast_days+1,np.mean(ending_vol),'r*')
        ax1.plot(forecast_days+1,min(np.max(ending_vol),np.mean(ending_vol) + 1 * np.std(ending_vol)),'r*')
        ax1.plot(forecast_days+1,max(np.min(ending_vol),np.mean(ending_vol) - 1 * np.std(ending_vol)),'r*')
        ax1.plot(forecast_days+1,np.max(ending_vol),'r*')
        ax1.plot(forecast_days+1,np.min(ending_vol),'r*')
        ax1.plot(forecast_days+1,new_std_data.iloc[forecast_days+1],'g*')
        ax2 = fig.add_subplot(614)
        ax2.plot((data/100+1).cumprod().tolist())
        ax3 = fig.add_subplot(615)
        ax3.plot(data.tolist())
        ax4 = fig.add_subplot(616)
        ax4.plot(all_std_data.diff().tolist())
        plt.show()
        break

In [ ]:
data.iloc[-100:].std()

In [ ]:
all_std_data.diff()

In [ ]:
while(True):
    stock = random.choice(full_cc.index.levels[0].tolist())
    data = full_cc.xs(stock,level=0)['close_to_close'].dropna()
    data = zz500
    std_data = data.rolling(window=100).std()
    if len(data) <= 600:
        print('too short')
        continue
    else:
        print(stock,len(data))
        band_up = []
        band_low = []
        band_up_lim = []
        band_low_lim = []
        band_mid = []
        for cut in range(len(data)-500,0,-1):
            new_std_data = std_data.iloc[-cut:]
            old_std_data = std_data.iloc[:-cut]
            all_std_data = old_std_data.dropna().copy()
            all_std_data.index = range(-len(all_std_data),0)
            all_std_data = all_std_data.append(new_std_data.reset_index(drop=True))
            old_std_data = old_std_data.dropna().tolist()
            sigma = np.std(old_std_data)*np.sqrt(252)
            m = 0.2
            theta_mean = np.mean(old_std_data)
            theta_min = max(np.min(old_std_data) - m * (np.max(old_std_data) - np.min(old_std_data)), volatility_data_min)
            theta_max = min(np.max(old_std_data) + m * (np.max(old_std_data) - np.min(old_std_data)), volatility_data_max)
            k = 2.5
            ending_vol = []
            forecast_days = 50
            for j in range(100):
                V_0 = new_std_data.iloc[0]
                V_n1 = old_std_data[-1]
                V_list = [V_n1,V_0]
                dt = 1/252
                
                for i in range(forecast_days):
                    V_old_old = V_list[-2]
                    V_old = V_list[-1]

                    choice = random.randint(0,1)
                    jump = random.randint(0,99)

                    m = 0.2
                    theta_min = max(np.min(old_std_data + V_list) - m * (np.max(old_std_data + V_list)\
                                                                         - np.min(old_std_data + V_list)), volatility_data_min)
                    theta_max = min(np.max(old_std_data + V_list) + m * (np.max(old_std_data + V_list)\
                                                                         - np.min(old_std_data + V_list)), volatility_data_max)

                    dist_to_max = (theta_max - max(V_old,theta_min))/(theta_max - theta_min)
                    dist_to_min = (min(V_old,theta_max) - theta_min)/(theta_max - theta_min)
                    dist_max_mid = (theta_max - theta_mean)/(theta_max - theta_min)
                    dist_min_mid = (theta_mean - theta_min)/(theta_max - theta_min)

                    if choice == 0:
                        V_new = V_old +\
                            np.abs(dist_to_max - dist_max_mid) * np.abs(dist_to_min - dist_min_mid) * (theta_mean - V_old) * dt * 4 * k+\
                            (dist_to_max * dist_to_min) * (V_old - V_old_old) * dt * 4 * k +\
                            sigma * (dist_to_max * dist_to_min * 100) * V_old * dt * k * k * dt
                    else:
                        V_new = V_old +\
                            np.abs(dist_to_max - dist_max_mid) * np.abs(dist_to_min - dist_min_mid) * (theta_mean - V_old) * dt * 4 * k +\
                            (dist_to_max * dist_to_min) * (V_old - V_old_old) * dt * 4 * k -\
                            sigma * (dist_to_max * dist_to_min * 100) * V_old * dt * k * k * dt

                    scaled_V_old = volatility_data_min + dist_to_min * (volatility_data_max - volatility_data_min)

                    if jump == 0:
                        jump_value = interval_to_percentage_distribution_df[next(col for col in interval_to_percentage_distribution_df\
                            .columns if col[0]>scaled_V_old)].loc[0]
                    elif jump == 1:
                        jump_value = interval_to_percentage_distribution_df[next(col for col in interval_to_percentage_distribution_df\
                            .columns if col[0]>scaled_V_old)].loc[n_interval-1]
                    else:
                        jump_value = 0

                    V_new += jump_value

                    V_list.append(V_new)
                ending_vol.append(V_new)
            band_mid.append(np.mean(ending_vol))
            band_up.append(np.max(ending_vol))
            band_low.append(np.min(ending_vol))
            band_up_lim.append(min(np.max(ending_vol),np.mean(ending_vol) + 1 * np.std(ending_vol)))
            band_low_lim.append(max(np.min(ending_vol),np.mean(ending_vol) - 1 * np.std(ending_vol)))
        plt.figure(figsize=(15,5))
        plt.plot(std_data.iloc[-(len(data)-(500 + forecast_days)):].tolist())
        plt.plot(band_mid,'g',alpha = 0.6)
        plt.plot(band_up,'r',alpha = 0.6)
        plt.plot(band_low,'r',alpha = 0.6)
        plt.plot(band_up_lim,'orange',alpha = 0.6)
        plt.plot(band_low_lim,'orange',alpha = 0.6)
        plt.show()
        break

In [ ]:
result = pd.concat([pd.Series(std_data.iloc[-(len(data)-(500 + forecast_days)):].tolist()),\
        pd.Series(band_mid),\
        pd.Series(band_up),\
        pd.Series(band_low),\
        pd.Series(band_up_lim),\
        pd.Series(band_low_lim)],axis=1)
result.columns = ['std','mid','up','low','midup','midlow']
result.to_csv('zz500stdforecast.csv')

In [ ]:
def std_band_future_value_df_generator(var_mid_list,var_up_list,var_low_list,ret_list,tw = 100):    
    var_mid_list.name = 'VaR_mid'
    var_up_list.name = 'VaR_up'
    var_low_list.name = 'VaR_low'
    var_to_test_data = pd.concat([var_mid_list.dropna(),var_up_list.dropna(),var_low_list.dropna()],axis = 1)
    
    for i in range(tw):
        var_to_test_data[i] = np.nan
    for i in var_to_test_data.index:
        test_data = ret_list.iloc[ret_list.index.tolist().index(i)+1:].iloc[:tw].reset_index(drop=True)
        if len(test_data) != tw:
            break
        for j in range(tw):
            var_to_test_data.loc[i,j] = test_data.loc[j]
            
    var_to_test_data.dropna(inplace=True)
    
    return var_to_test_data

In [ ]:
def std_band_count_exception(df, tw, VaR_type):
    VaR = df.loc[VaR_type]
    test_data = df.loc[range(tw)]
    
    return len(test_data[test_data<VaR])

In [ ]:
def std_band_kupiec_backtest(df, cl, tw, exception_type):
    cl_a = 1 - cl
    
    LRuc_left = -2*np.log((cl**(tw - df.loc[exception_type])) * (cl_a**df.loc[exception_type]))
    LRuc_right = 2*np.log(((1 - df.loc[exception_type]/tw)**(tw - df.loc[exception_type]) *\
                           ((df.loc[exception_type]/tw)**df.loc[exception_type])))
    
    return LRuc_left + LRuc_right

In [ ]:
def std_band_christoffersen_test(df, tw, VaR_type, exception_type):
    VaR = df.loc[VaR_type]
    n_exception = df.loc[exception_type]
    if n_exception == 0 or (n_exception == 1 and df.loc[tw-1] < VaR):
        return 0
    
    test_data = df.loc[range(tw)].tolist()
    
    n_00 = len([value for index,value in enumerate(test_data[:-1]) if value > VaR and test_data[index+1] > VaR])
    n_10 = len([value for index,value in enumerate(test_data[:-1]) if value < VaR and test_data[index+1] > VaR])
    n_01 = len([value for index,value in enumerate(test_data[:-1]) if value > VaR and test_data[index+1] < VaR])
    n_11 = len([value for index,value in enumerate(test_data[:-1]) if value < VaR and test_data[index+1] < VaR])
    
    pi_0 = n_01 / (n_00 + n_01)
    pi_1 = n_11 / (n_10 + n_11)
    pi = (n_01 + n_11) / (n_00 + n_01 + n_10 + n_11)
    
    LRcc_up = ((1 - pi)**(n_00 + n_10)) * (pi**(n_01+n_11))
    LRcc_down = ((1 - pi_0)**n_00) * (pi_0**n_01) * ((1 - pi_1)**n_10) * (pi_1**n_11)
    LRcc = -2*np.log(LRcc_up / LRcc_down)
    return LRcc

In [ ]:
def std_band_exception_test(df, cl, uc_type, cc_type):
    LRuc = df.loc[uc_type]
    LRcc = df.loc[cc_type]
    
    #uc - chi-sqaure 1 df
    #cc - chi-square 2 df
    LRuc_limit = stats.chi2.ppf(cl, df = 1)
    LRcc_limit = stats.chi2.ppf(cl, df = 2)
    
    if LRcc > LRcc_limit:
        if LRuc + LRcc > LRuc_limit + LRcc_limit:
            return 'reject'
        else:
            return 'accept'
    else:
        if LRuc > LRuc_limit:
            return 'reject'
        else:
            return 'accept'

In [ ]:
def std_band_backtest_VaR(ret_data, var_mid_data, var_up_data, var_low_data, window = 100, var_cl = 0.95, test_cl = 0.95):
    var_mid_data.name = 'VaR_mid'
    var_up_data.name = 'VaR_up'
    var_low_data.name = 'VaR_low'

    var_confidence_level = var_cl
    test_confidence_level = test_cl
    
    data =  std_band_future_value_df_generator(var_mid_data,var_up_data,var_low_data,ret_data,tw = window)
    data['n_exception_VaR_mid'] = data.apply(std_band_count_exception,args=(window,'VaR_mid',),axis=1)
    data['n_exception_VaR_up'] = data.apply(std_band_count_exception,args=(window,'VaR_up',),axis=1)
    data['n_exception_VaR_low'] = data.apply(std_band_count_exception,args=(window,'VaR_low',),axis=1)
    data['LRuc_VaR_mid'] = data.apply(std_band_kupiec_backtest,args=(var_confidence_level,window,'n_exception_VaR_mid',),axis=1)
    data['LRuc_VaR_up'] = data.apply(std_band_kupiec_backtest,args=(var_confidence_level,window,'n_exception_VaR_up',),axis=1)
    data['LRuc_VaR_low'] = data.apply(std_band_kupiec_backtest,args=(var_confidence_level,window,'n_exception_VaR_low',),axis=1)
    data['LRcc_VaR_mid'] = data.apply(std_band_christoffersen_test,args=(window,'VaR_mid','n_exception_VaR_mid',),axis=1)
    data['LRcc_VaR_up'] = data.apply(std_band_christoffersen_test,args=(window,'VaR_up','n_exception_VaR_up',),axis=1)
    data['LRcc_VaR_low'] = data.apply(std_band_christoffersen_test,args=(window,'VaR_low','n_exception_VaR_low',),axis=1)
    data['result_VaR_mid'] = data.apply(std_band_exception_test,args=(test_confidence_level,'LRuc_VaR_mid','LRcc_VaR_mid',),axis=1)
    data['result_VaR_up'] = data.apply(std_band_exception_test,args=(test_confidence_level,'LRuc_VaR_up','LRcc_VaR_up',),axis=1)
    data['result_VaR_low'] = data.apply(std_band_exception_test,args=(test_confidence_level,'LRuc_VaR_low','LRcc_VaR_low',),axis=1)
    
    return data

In [ ]:
vaV = pd.DataFrame([std_data.iloc[-(len(data)-500):].tolist(),band_up,band_mid,band_low],index=['std','upper','mid','lower'],\
                   columns=std_data.index.tolist()[-len(data)+500:]).T

In [ ]:
vaV['ret'] = data.loc[vaV.index]

In [ ]:
vaV

In [ ]:
test_window = 100

var_confidence_level = 0.95
test_confidence_level = 0.95

In [ ]:
index_series = []
VaR_series = []
param_lambda = 0.94
param_max_length = 200
VaR = None
old_VaR = None
for i in range(len(vaV)):
    sub_vaV = vaV.iloc[:i+1].copy()
    if len(sub_vaV) < test_window:
        continue
    if len(sub_vaV) <= param_max_length:
        pass
    else:
        sub_vaV = sub_vaV.iloc[-param_max_length:]
    newest_forecasted_vol = sub_vaV.iloc[-1,1]
    sub_vaV['adjusted_ret'] = sub_vaV['ret']*((newest_forecasted_vol/sub_vaV['std']))
    index_series.append(sub_vaV.index.tolist()[-1])
    bootstrap_factor = 0.6
    VaR_sub_list = []
    for k in range(10):
        selecting = random.sample([j for j in range(len(sub_vaV))],int(len(sub_vaV)*bootstrap_factor))
        sub_sub_vaV = sub_vaV.iloc[selecting]
        sub_sub_vaV['weights'] = [(param_lambda**(j-1))*(1-param_lambda)/(1-param_lambda**len(sub_sub_vaV)) \
            for j in range(len(sub_sub_vaV))][::-1]
        sub_sub_vaV = sub_sub_vaV.sort_values('adjusted_ret')
        sub_sub_vaV['weights_cumsum'] = sub_sub_vaV['weights'].cumsum()
        VaR = sub_sub_vaV['adjusted_ret'].iloc[next(index for index, value in enumerate(sub_sub_vaV['weights_cumsum']) \
                                                        if value > (1-var_confidence_level))]
        VaR_sub_list.append(VaR)
    VaR = np.mean(VaR_sub_list)
    if old_VaR == None:
        old_VaR = VaR
        VaR_series.append(VaR)
        continue
    else:
        if VaR > 0.99 * old_VaR:
            VaR = 0.99 * old_VaR
    old_VaR = VaR
    VaR_series.append(VaR)
exp_var_up = pd.Series(VaR_series,index=index_series)
exp_var_up = pd.ewma(exp_var_up,span = 20)

In [ ]:
index_series = []
VaR_series = []
param_lambda = 0.94
param_max_length = 200
VaR = None
old_VaR = None
for i in range(len(vaV)):
    sub_vaV = vaV.iloc[:i+1].copy()
    if len(sub_vaV) < test_window:
        continue
    if len(sub_vaV) <= param_max_length:
        pass
    else:
        sub_vaV = sub_vaV.iloc[-param_max_length:]
    newest_forecasted_vol = sub_vaV.iloc[-1,0]
    sub_vaV['adjusted_ret'] = sub_vaV['ret']*((newest_forecasted_vol/sub_vaV['std']))
    index_series.append(sub_vaV.index.tolist()[-1])
    bootstrap_factor = 0.6
    VaR_sub_list = []
    for k in range(10):
        selecting = random.sample([j for j in range(len(sub_vaV))],int(len(sub_vaV)*bootstrap_factor))
        sub_sub_vaV = sub_vaV.iloc[selecting]
        sub_sub_vaV['weights'] = [(param_lambda**(j-1))*(1-param_lambda)/(1-param_lambda**len(sub_sub_vaV)) \
            for j in range(len(sub_sub_vaV))][::-1]
        sub_sub_vaV = sub_sub_vaV.sort_values('adjusted_ret')
        sub_sub_vaV['weights_cumsum'] = sub_sub_vaV['weights'].cumsum()
        VaR = sub_sub_vaV['adjusted_ret'].iloc[next(index for index, value in enumerate(sub_sub_vaV['weights_cumsum']) \
                                                        if value > (1-var_confidence_level))]
        VaR_sub_list.append(VaR)
    VaR = np.mean(VaR_sub_list)
    if old_VaR == None:
        old_VaR = VaR
        VaR_series.append(VaR)
        continue
    else:
        if VaR > 0.99 * old_VaR:
            VaR = 0.99 * old_VaR
    old_VaR = VaR
    VaR_series.append(VaR)
exp_var_mid = pd.Series(VaR_series,index=index_series)
exp_var_mid = pd.ewma(exp_var_mid,span = 20)

In [ ]:
index_series = []
VaR_series = []
param_lambda = 0.94
param_max_length = 200
VaR = None
old_VaR = None
for i in range(len(vaV)):
    sub_vaV = vaV.iloc[:i+1].copy()
    if len(sub_vaV) < test_window:
        continue
    if len(sub_vaV) <= param_max_length:
        pass
    else:
        sub_vaV = sub_vaV.iloc[-param_max_length:]
    newest_forecasted_vol = sub_vaV.iloc[-1,3]
    sub_vaV['adjusted_ret'] = sub_vaV['ret']*((newest_forecasted_vol/sub_vaV['std']))
    index_series.append(sub_vaV.index.tolist()[-1])
    bootstrap_factor = 0.6
    VaR_sub_list = []
    for k in range(10):
        selecting = random.sample([j for j in range(len(sub_vaV))],int(len(sub_vaV)*bootstrap_factor))
        sub_sub_vaV = sub_vaV.iloc[selecting]
        sub_sub_vaV['weights'] = [(param_lambda**(j-1))*(1-param_lambda)/(1-param_lambda**len(sub_sub_vaV)) \
            for j in range(len(sub_sub_vaV))][::-1]
        sub_sub_vaV = sub_sub_vaV.sort_values('adjusted_ret')
        sub_sub_vaV['weights_cumsum'] = sub_sub_vaV['weights'].cumsum()
        VaR = sub_sub_vaV['adjusted_ret'].iloc[next(index for index, value in enumerate(sub_sub_vaV['weights_cumsum']) \
                                                        if value > (1-var_confidence_level))]
        VaR_sub_list.append(VaR)
    VaR = np.mean(VaR_sub_list)
    if old_VaR == None:
        old_VaR = VaR
        VaR_series.append(VaR)
        continue
    else:
        if VaR > 0.99 * old_VaR:
            VaR = 0.99 * old_VaR
    old_VaR = VaR
    VaR_series.append(VaR)
exp_var_low = pd.Series(VaR_series,index=index_series)
exp_var_low = pd.ewma(exp_var_low,span = 20)

In [ ]:
backtest = std_band_backtest_VaR(data, exp_var_mid, exp_var_up, exp_var_low, window = 50, var_cl = 0.95, test_cl = 0.95)

In [ ]:
pd.set_option('display.max_rows',20)
pd.set_option('display.max_columns',30)

In [ ]:
final_result = []
for i in backtest.index:
    if backtest.loc[i,'result_VaR_mid'] == 'accept' \
        or backtest.loc[i,'result_VaR_up'] == 'accept' \
        or backtest.loc[i,'result_VaR_low'] == 'accept':
        final_result.append('accept')
    else:
        final_result.append('reject')
backtest['final_result'] = final_result

In [ ]:
backtest['final_result'].value_counts()/len(backtest)

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(111)
ax1.plot(backtest['VaR_mid'].tolist())
ax1.set_ylabel('VaR_mid')

ax2 = ax1.twinx()  # this is the important function
ax2.plot(backtest['n_exception_VaR_mid'].tolist(),'r.')
ax2.hlines(0.5,0,len(backtest))
ax2.hlines(6.5,0,len(backtest))
ax2.set_ylabel('n_exception_VaR_mid')
plt.show()
plt.plot((data.loc[backtest.index]/100+1).cumprod().tolist())
plt.ylabel('stock')
plt.show()

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(111)
ax1.plot(backtest['VaR_up'].tolist())
ax1.set_ylabel('VaR_up')

ax2 = ax1.twinx()  # this is the important function
ax2.plot(backtest['n_exception_VaR_up'].tolist(),'r.')
ax2.hlines(0.5,0,len(backtest))
ax2.hlines(6.5,0,len(backtest))
ax2.set_ylabel('n_exception_VaR_up')
plt.show()
plt.plot((data.loc[backtest.index]/100+1).cumprod().tolist())
plt.ylabel('stock')
plt.show()

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(111)
ax1.plot(backtest['VaR_low'].tolist())
ax1.set_ylabel('VaR_low')

ax2 = ax1.twinx()  # this is the important function
ax2.plot(backtest['n_exception_VaR_low'].tolist(),'r.')
ax2.hlines(0.5,0,len(backtest))
ax2.hlines(6.5,0,len(backtest))
ax2.set_ylabel('n_exception_VaR_low')
plt.show()
plt.plot((data.loc[backtest.index]/100+1).cumprod().tolist())
plt.ylabel('stock')
plt.show()

In [ ]:
backtest